# 1 Data Collection

1.1 Set-up Environment

In [ ]:
import requests
import json
import pandas as pd

1.2 Condition for sending a request and establishing variables to store data.

In [ ]:
extensions = '%7B%22persistedQuery%22%3A%7B%22sha256Hash%22%3A%227327d144ec84b57c93f761affe0d0609b0d495f85e8e47fdc76291679850cfda%22%2C%22version%22%3A1%7D%7D'


url = 'https://caching.graphql.imdb.com/?operationName=AdvancedTitleSearch&variables={}&extensions={}'
# url = 'https://caching.graphql.imdb.com/'

headers = {
    "authority":"caching.graphql.imdb.com",
    "accept":"application/graphql+json, application/json",
    "accept-language":"en-US,en;q=0.9",
    "content-type":"application/json",
    "cookie":"uu=eyJpZCI6InV1ZjdkZDIxZTQxNjI2NDk5ZDlmMzQiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; session-id=130-5051456-6343236; ubid-main=135-3141743-8299733; session-id-time=2082787201l; session-token=gCSbg4aEjMHkKMQozdNXcZ9pw5Bx1pDKZO/E/hxf99m9bObwEAHonk2ZLiMp+/1y9+pRdPmKxBz5OKrKF7h/kTuTduzH2HavA4pchVBWbVvefQHVOCICwLLzTplkJexUZWnijHssy5Rrr3vonX332+wJtjUll2E4RuXlAVpq6UMLDgDdi11TOX4Nc4FTmxJFDNXpXLF0VUii18SLtS37GHC5zKNnsv/3YWVevSV+jjDXuUV9nYtmwY7yCUhfqJI2nukbbbXygVPwbdGLRh3WQAJr9jOpGrN+2HV1IgwHrsxDuj4wF1NNiL0jjg+zuWKB7BVbGTVQuTLpQXHGLO4WHkYwZoBCZf0r; ad-oo=0",
    "origin":"https://www.imdb.com",
    "referer":"https://www.imdb.com/",
    "sec-fetch-dest":"empty",
    "sec-fetch-mode":"cors",
    "sec-fetch-site":"same-site",
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.118 Safari/537.36",
    "x-amzn-sessionid":"130-5051456-6343236",
    "x-imdb-client-name":"imdb-web-next-localized",
    "x-imdb-client-rid":"16Q3WHX7RCCGW6H7YS4C",
    "x-imdb-user-country":"US",
    "x-imdb-user-language":"en-US",
    "x-imdb-weblab-treatment-overrides":'{"IMDB_DESKTOP_SEARCH_ALGORITHM_UPDATES_577300":"T1"}',
}

full_data = {}
full_data.setdefault('Title', [])
full_data.setdefault('Genre',[])
full_data.setdefault('ReleaseYear',[])
full_data.setdefault('Rating',[])
full_data.setdefault('VoteCount',[])
full_data.setdefault('Duration',[])
full_data.setdefault('Age Limit',[])
year = range(2014, 2023)

1.3 Function

In [ ]:
def parse_value(data):
    #Name
    full_data['Title'].append(data['node']['title']['titleText']['text'])
    
    #Genres
    GenreList=[]
    for genre in data['node']['title']['titleGenres']['genres']:
        GenreList.append(genre['genre']['text'])
    full_data['Genre'].append(GenreList)
    
    #ReleaseYear
    full_data['ReleaseYear'].append(data['node']['title']['releaseYear']['year'])
    
    #Rating and RatingCount
    full_data['Rating'].append(data['node']['title']['ratingsSummary']['aggregateRating'])
    full_data['VoteCount'].append(data['node']['title']['ratingsSummary']['voteCount'])

    #Duration
    if data['node']['title']['runtime'] is not None:
        full_data['Duration'].append(data['node']['title']['runtime']['seconds']/3600)
    else: full_data['Duration'].append(None)
    
    #AgeLimit
    if data['node']['title']['certificate'] is not None:
        full_data['Age Limit'].append(data['node']['title']['certificate']['rating'])
    else : full_data['Age Limit'].append(None)
        
        
def generate_variable(genre, from_time, to_time):
    return json.dumps({"certificateConstraint":{},"colorationConstraint":{},"creditedCompanyConstraint":{},"first":50,"genreConstraint":{"allGenreIds":[genre]},"listConstraint":{"inAllLists":[],"inAllPredefinedLists":[],"notInAnyList":[],"notInAnyPredefinedList":[]},"locale":"en-US","releaseDateConstraint":{"releaseDateRange":{"end":f"{to_time}-01-01","start":f"{from_time}-01-01"}},"runtimeConstraint":{"runtimeRangeMinutes":{}},"sortBy":"POPULARITY","sortOrder":"ASC","titleTypeConstraint":{"anyTitleTypeIds":["movie"]},"userRatingsConstraint":{"aggregateRatingRange":{},"ratingsCountRange":{}}})

# def Filter_duplicate_data():
#     df = pd.DataFrame(full_data)
#     df.drop_duplicates(subset=["Title","ReleaseYear"])

In [ ]:
for from_time, to_time in zip(year[:-1], year[1:]):
    for genre in ['Action', 'Adventure', 'Animation','Comedy','Crime']:
        params = {
                    'operationName' : 'AdvancedTitleSearch',
                    'variables' : generate_variable(genre, from_time, to_time),
                    'extensions' : extensions
                    }
        data = requests.get(url=url.format(params['variables'],params['extensions']), headers=headers)
        # data = requests.get(url=url, params=params, headers=headers)
        a = data.json()
        for data in a['data']['advancedTitleSearch']['edges']:
            parse_value(data)
#new_data= Filter_duplicate_data()
#new_data.to_csv('output.csv', index=False)
full_data.to_csv('output.csv',index=False)